make figure showing:

- Merapi true colour
- Merapi slope
- Merapi shadow overlay map
- interferogram
- pixel offset range and azimuth


In [8]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [9]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# Indonesia shapefile
IND_COAST = './test_data/DEM/IDN_adm/IDN_adm0.shp' 

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'



# DEM, SLOPE and NDVI file
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'

IFG_FILE1 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif'
HEADING = 168
INC_ANGLE = 35.6
# lava flow files:
L1888_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1948_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp'
L1956_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp'
L1992_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1997_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp'
L1998_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'
CRATER_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp'
# define map region of interest
lon_lims = [110.41, 110.45]
lat_lims = [-7.555, -7.535]


# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]


# get world borders via geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))



In [10]:
import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values


def read_tiff(filename,n_bands):
    with rio.open(filename) as src:

        values = [src.read(i+1) for i in range(n_bands)]
        if n_bands > 1:
            values_stack = np.stack(values,axis=-1)
        else:
            values_stack = values[0]
        print(f'Data {filename} has shape', values_stack.shape)
        height = values_stack.shape[0]
        width = values_stack.shape[1]
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rio.transform.xy(src.transform, rows, cols)
        lons = np.array(xs)
        lats = np.array(ys)
        extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]
    return values_stack, lons, lats, extent 


# load indonesia borders
ind_coastlines = gpd.read_file(IND_COAST)


DEM_heights, DEM_lons, DEM_lats, DEM_extent = read_tiff(DEM_FILE,1)
DEM_slope, slope_lons, slope_lats, slope_extent = read_tiff(SLOPE_FILE,1)

## NDVI open and crop data to DEM extent
# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]

# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.53]

CSK_IFG1, CSK_IFG_lons1, CSK_IFG_lats1, CSK_IFG_extent1 = read_tiff(IFG_FILE1,3)


x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Data ./test_data/DEM/TDX_Merapi_WGS84_5m.tif has shape (2028, 1672)
Data ./test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif has shape (2028, 1672)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif has shape (10171, 8384, 3)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_71082/812364364.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_71082/812364364.py:46: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [11]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

from matplotlib.colors import LinearSegmentedColormap
clist = ['#f579cd', '#f67fc6', '#f686bf', '#f68cb9', '#f692b3', '#f698ad',
                     '#f69ea7', '#f6a5a1', '#f6ab9a', '#f6b194', '#f6b78e', '#f6bd88',
                     '#f6c482', '#f6ca7b', '#f6d075', '#f6d66f', '#f6dc69', '#f6e363',
                     '#efe765', '#e5eb6b', '#dbf071', '#d0f477', '#c8f67d', '#c2f684',
                     '#bbf68a', '#b5f690', '#aff696', '#a9f69c', '#a3f6a3', '#9cf6a9',
                     '#96f6af', '#90f6b5', '#8af6bb', '#84f6c2', '#7df6c8', '#77f6ce',
                     '#71f6d4', '#6bf6da', '#65f6e0', '#5ef6e7', '#58f0ed', '#52e8f3',
                     '#4cdbf9', '#7bccf6', '#82c4f6', '#88bdf6', '#8eb7f6', '#94b1f6',
                     '#9aabf6', '#a1a5f6', '#a79ef6', '#ad98f6', '#b392f6', '#b98cf6',
                     '#bf86f6', '#c67ff6', '#cc79f6', '#d273f6', '#d86df6', '#de67f6',
                     '#e561f6', '#e967ec', '#ed6de2', '#f173d7']
colormap = LinearSegmentedColormap.from_list('dismph', clist, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

dismph = colormap(range(255))
dismph2 = np.row_stack((np.array(dismph),np.array(dismph)))
print(dismph2)

colormap = LinearSegmentedColormap.from_list('dismph2', dismph2, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

[[0.96078431 0.4745098  0.80392157 1.        ]
 [0.96175317 0.48032295 0.79713956 1.        ]
 [0.96272203 0.4861361  0.79035755 1.        ]
 ...
 [0.93347174 0.43354095 0.87510957 1.        ]
 [0.93734717 0.43935409 0.86445213 1.        ]
 [0.94122261 0.44516724 0.85379469 1.        ]]


In [12]:
%matplotlib osx
## overview figure

# define path to shapefile

# Read the shapefile
L1888 = gpd.read_file(L1888_FILE)
L1948 = gpd.read_file(L1948_FILE)
L1956 = gpd.read_file(L1956_FILE)
L1992 = gpd.read_file(L1992_FILE)
L1997 = gpd.read_file(L1997_FILE)
L1998 = gpd.read_file(L1998_FILE)
CRATER = gpd.read_file(CRATER_FILE)

# Extract latitude and longitude into separate columns
coords_L1888 = np.array(list(L1888["geometry"][0].coords))
coords_L1956_1 = np.array(list(L1956["geometry"][0].coords))
coords_L1956_2 = np.array(list(L1956["geometry"][1].coords))
coords_L1948 = np.array(list(L1948["geometry"][0].coords))
coords_L1992 = np.array(list(L1992["geometry"][0].coords))
coords_L1997 = np.array(list(L1997["geometry"][0].coords))
coords_L1998 = np.array(list(L1998["geometry"][0].coords))
coords_CRATER = np.array(list(CRATER["geometry"][0].coords))


In [13]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

def add_bottom_cax(fig, ax_l, ax_r, pad, height):
    axpos_l = ax_l.get_position()
    axpos_r = ax_r.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos_l.x0,
        axpos_l.y0 - pad -height,
        axpos_r.x1,
        axpos_r.y0 - pad
    )
    cax = fig.add_axes(caxpos)

    return cax

def draw_sat_arrow(ax,arrow_start,azimuth,look_dir = 'right',inc_angle = 90,arrow_length=0.001, arrow_width = 0.0001,color='black',fontsize=16,text_offset=0.002):
    ax.arrow(arrow_start[0],arrow_start[1],
            np.cos(np.radians(azimuth+90))*arrow_length,
            np.sin(np.radians(azimuth+90))*arrow_length,
            color=color,width=arrow_width)
    ax.arrow(arrow_start[0]+0.5*np.cos(np.radians(azimuth+90))*arrow_length,
            arrow_start[1]+0.5*np.sin(np.radians(azimuth+90))*arrow_length,
            np.cos(np.radians(azimuth))*arrow_length*0.5,
            np.sin(np.radians(azimuth))*arrow_length*0.5,
            color=color,width=arrow_width)
    if inc_angle != 90:
        ax.annotate(f'{inc_angle}°',xy=[arrow_start[0]-text_offset,arrow_start[1]+0.1*text_offset], xycoords='data',color=color,fontsize=fontsize)

        


In [17]:
fig = plt.figure(figsize=(8,8))

gs = GridSpec(2, 2, figure=fig,height_ratios=[10,5.2])
ax0a = fig.add_subplot(gs[0, :]) # Merapi Interferogram
ax10 = fig.add_subplot(gs[1, 0]) # zoom 1
ax11 = fig.add_subplot(gs[1, 1]) # zoom 2

# plot extents
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.53]

lon_lims_z1 = [110.427, 110.436]
lat_lims_z1 = [-7.545, -7.537]

lon_lims_z2 = [110.438, 110.45]
lat_lims_z2 = [-7.541, -7.534]

ax0a.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax0a.set_xlim(lon_lims)
ax0a.set_ylim(lat_lims)

ax0a.plot([lon_lims_z1[0], lon_lims_z1[1],lon_lims_z1[1],lon_lims_z1[0],lon_lims_z1[0]],
        [lat_lims_z1[0], lat_lims_z1[0],lat_lims_z1[1],lat_lims_z1[1],lat_lims_z1[0]],
        linewidth=2,
        color='red')
ax0a.plot([lon_lims_z2[0], lon_lims_z2[1],lon_lims_z2[1],lon_lims_z2[0],lon_lims_z2[0]],
        [lat_lims_z2[0], lat_lims_z2[0],lat_lims_z2[1],lat_lims_z2[1],lat_lims_z2[0]],
        linewidth=2,
        color='red')


# zoom 1

ax10.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax10.set_xlim(lon_lims_z1)
ax10.set_ylim(lat_lims_z1)

# zoom 2

ax11.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax11.set_xlim(lon_lims_z2)
ax11.set_ylim(lat_lims_z2)

bbox = dict(fc="1.0")
bbox2 = dict(fc="1.0",alpha=0.7)
for a in [ax0a,ax10,ax11]:
    # a.set_aspect('equal','box')
    # a.annotate(l,xy=(0.93,0.83),xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper right',font_properties={
        "family": "serif",
        "size": 16,
    }))
    # a.set_xlim(zoom_lon_lims)
    # a.set_ylim(zoom_lat_lims)
    a.set_axis_off()
ax0a.legend(loc=4,title='Lava flows')
fig.subplots_adjust(hspace=0.01, wspace = 0.01)

ax0a_pos = ax0a.get_position()
ax10_pos = ax10.get_position()
ax11_pos = ax11.get_position()

ax10.set_position([ax0a_pos.x0,
                   ax10_pos.y0,
                   ax10_pos.width,
                   ax10_pos.height])

ax11.set_position([ax0a_pos.x1-ax11_pos.width,
                   ax10_pos.y1-ax11_pos.height,
                   ax11_pos.width,
                   ax11_pos.height])

draw_sat_arrow(ax0a,[110.457,-7.535],HEADING,look_dir = 'right',inc_angle = INC_ANGLE,arrow_length=0.0025, arrow_width = 0.0002,color='white',fontsize=16,text_offset=0.002)
ax0a.annotate('A',xy=(0.01,0.943),xycoords='axes fraction',fontsize=16, bbox=bbox)
ax0a.annotate('B',xy=(0.046,0.7),xycoords='axes fraction',fontsize=16, bbox=bbox2)
ax0a.annotate('C',xy=(0.36,0.82),xycoords='axes fraction',fontsize=16, bbox=bbox2)

ax10.annotate('B',xy=(0.025,0.89),xycoords='axes fraction',fontsize=16, bbox=bbox)
ax11.annotate('C',xy=(0.02,0.87),xycoords='axes fraction',fontsize=16, bbox=bbox)

fake_data = ax0a.imshow(np.random.random((10,2)),cmap=colormap, vmin=-1,vmax=1,alpha = 1)

cax = add_bottom_cax(fig,ax11,ax11,0.01,0.02)
cbar0 = fig.colorbar(fake_data, cax=cax, orientation='horizontal',)
ticklabs = cbar0.ax.get_yticklabels()
cbar0.ax.set_yticklabels(ticklabs, fontsize=16)
cbar0.set_label( label='LOS change [cm]', fontsize=16)
cbar0.set_ticks(ticks=[-1, 0, 1], labels=[r'-1.55', '0', r'+1.55'],fontsize=16)

fig.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/Merapi_ifg.pdf',dpi=300)


: 

In [68]:
plt.close('all')